## DIA 021:  Implementación de Pruebas Automatizadas y Configuración de CI/CD para la API de Predicción

# **Resumen del Día 21: Pruebas Automatizadas y Configuración de CI/CD para la API de Predicción**

## **Objetivos del Día 21**
1. **Implementar Pruebas Automatizadas:**
   - Asegurar la calidad y correcto funcionamiento de la API mediante pruebas unitarias e integradas.
   
2. **Configurar un Pipeline de CI/CD:**
   - Automatizar el proceso de pruebas y despliegue para mejorar la eficiencia y reducir errores manuales.
   
3. **Integrar Pruebas y CI/CD con Heroku:**
   - Garantizar que cada cambio en el código pase por pruebas antes de ser desplegado automáticamente en Heroku.

---

## **1. Implementar Pruebas Automatizadas**

### **1.1. Instalación de Dependencias para Pruebas**
- **Herramientas Utilizadas:**
  - **PyTest:** Framework de pruebas para Python.
  - **pytest-flask:** Extensión de PyTest para facilitar las pruebas de aplicaciones Flask.

**Actualizar `requirements.txt`:**
```plaintext
Flask==2.0.3
tensorflow==2.9.1
Pillow==9.0.1
numpy==1.23.1
scikit-learn==1.1.1
gunicorn==20.1.0
loguru==0.6.0
prometheus_flask_exporter==0.20.0
flask-swagger-ui==4.1.0
pytest==7.1.2
pytest-flask==1.2.0
```

**Instalar las Dependencias:**
```bash
pip install -r requirements.txt
```

### **1.2. Crear Pruebas Unitarias para la API**

**Estructura de Pruebas:**
- Crear un directorio `tests` con archivos de pruebas como `test_api.py`.

**Escribir Pruebas en `test_api.py`:**
```python
# tests/test_api.py

import pytest
from api import app
import os
from io import BytesIO
from PIL import Image
import numpy as np

@pytest.fixture
def client():
    app.config['TESTING'] = True
    with app.test_client() as client:
        yield client

def generate_image(digit=5):
    # Genera una imagen en blanco con un dígito negro en el centro
    img = Image.new('L', (28, 28), color=255)
    # Aquí podrías dibujar el dígito, pero para simplicidad, dejaremos la imagen en blanco
    # En un caso real, usarías una imagen real de MNIST
    return img

def test_predict_success(client):
    # Obtener la API Key desde variables de entorno
    api_key = os.getenv('API_KEY')
    assert api_key is not None, "API_KEY no está configurada en las variables de entorno."
    
    img = generate_image()
    img_byte_arr = BytesIO()
    img.save(img_byte_arr, format='PNG')
    img_byte_arr.seek(0)
    
    data = {
        'file': (img_byte_arr, 'test.png')
    }
    headers = {
        'x-api-key': api_key
    }
    
    response = client.post('/predict', data=data, headers=headers, content_type='multipart/form-data')
    assert response.status_code == 200
    json_data = response.get_json()
    assert 'prediccion' in json_data
    assert 'probabilidad' in json_data
    assert isinstance(json_data['prediccion'], int)
    assert isinstance(json_data['probabilidad'], float)

def test_predict_no_file(client):
    api_key = os.getenv('API_KEY')
    assert api_key is not None, "API_KEY no está configurada en las variables de entorno."
    
    data = {}
    headers = {
        'x-api-key': api_key
    }
    
    response = client.post('/predict', data=data, headers=headers, content_type='multipart/form-data')
    assert response.status_code == 400
    json_data = response.get_json()
    assert json_data['error'] == 'No se encontró el archivo en la solicitud.'

def test_predict_invalid_file_type(client):
    api_key = os.getenv('API_KEY')
    assert api_key is not None, "API_KEY no está configurada en las variables de entorno."
    
    # Crear un archivo de texto en lugar de una imagen
    file_data = BytesIO(b"Este no es una imagen válida.")
    data = {
        'file': (file_data, 'test.txt')
    }
    headers = {
        'x-api-key': api_key
    }
    
    response = client.post('/predict', data=data, headers=headers, content_type='multipart/form-data')
    assert response.status_code == 400
    json_data = response.get_json()
    assert json_data['error'] == 'Tipo de archivo no permitido. Por favor, sube una imagen PNG, JPG, JPEG o GIF.'

def test_predict_unauthorized(client):
    # No proporcionar la API Key
    img = generate_image()
    img_byte_arr = BytesIO()
    img.save(img_byte_arr, format='PNG')
    img_byte_arr.seek(0)
    
    data = {
        'file': (img_byte_arr, 'test.png')
    }
    
    response = client.post('/predict', data=data, content_type='multipart/form-data')
    assert response.status_code == 401
    json_data = response.get_json()
    assert json_data['error'] == 'Unauthorized access'
```

**Explicación:**
- **Fixture `client`:** Configura un cliente de pruebas para la aplicación Flask.
- **Función `generate_image`:** Genera una imagen de prueba.
- **Pruebas:**
  - **`test_predict_success`:** Verifica que una solicitud válida con una imagen y una API Key correcta devuelve una predicción exitosa.
  - **`test_predict_no_file`:** Verifica que una solicitud sin archivo devuelve un error adecuado.
  - **`test_predict_invalid_file_type`:** Verifica que subir un archivo no permitido (e.g., `.txt`) devuelve un error adecuado.
  - **`test_predict_unauthorized`:** Verifica que una solicitud sin la API Key correcta devuelve un error de acceso no autorizado.

### **1.3. Ejecutar las Pruebas Localmente**

**Comando para Ejecutar Pruebas:**
```bash
pytest
```

**Resultados Esperados:**
```
============================= test session starts ==============================
platform linux -- Python 3.9.1, pytest-7.1.2, pluggy-1.0.0
rootdir: /path/to/your/project
collected 5 items

tests/test_api.py .....                                             [100%]

============================== 5 passed in 0.50s ===============================
```

**Solución de Problemas:**
- **Error: `API_KEY no está configurada en las variables de entorno.`**
  - Asegúrate de haber exportado la variable `API_KEY` correctamente.
  
- **Error al Cargar el Modelo:**
  - Verifica que el archivo `modelo_cnn_transfer_learning_mnist_final_optimizado.h5` esté presente y accesible.
  
- **Errores de Dependencias:**
  - Asegúrate de haber instalado todas las dependencias listadas en `requirements.txt`.

---

## **2. Configurar un Pipeline de CI/CD con GitHub Actions**

La integración continua (CI) y el despliegue continuo (CD) automatizan el proceso de pruebas y despliegue, asegurando que cada cambio en el código pase por pruebas antes de ser desplegado automáticamente en Heroku.

### **2.1. Crear el Archivo de Workflow de GitHub Actions**

**Ubicación:** `.github/workflows/ci-cd.yml`

**Contenido Principal:**
```yaml
# .github/workflows/ci-cd.yml

name: CI/CD Pipeline

on:
  push:
    branches: [ master ]
  pull_request:
    branches: [ master ]

jobs:
  build-and-test:
    runs-on: ubuntu-latest

    steps:
    - name: Checkout Repository
      uses: actions/checkout@v3

    - name: Set up Python
      uses: actions/setup-python@v4
      with:
        python-version: '3.9'

    - name: Install Dependencies
      run: |
        python -m pip install --upgrade pip
        pip install -r requirements.txt

    - name: Run Tests
      run: |
        export API_KEY=e3b0c44298fc1c149afbf4c8996fb924  # Reemplaza con tu API Key real o usa secretos
        pytest

  deploy:
    needs: build-and-test
    runs-on: ubuntu-latest

    steps:
    - name: Checkout Repository
      uses: actions/checkout@v3

    - name: Login to Heroku Container Registry
      uses: akshnz/heroku-docker-login@v1.3.1
      with:
        email: ${{ secrets.HEROKU_EMAIL }}
        api_key: ${{ secrets.HEROKU_API_KEY }}

    - name: Build, Push and Release Docker image to Heroku
      run: |
        heroku container:push web --app mnist-flask-api
        heroku container:release web --app mnist-flask-api
```

**Explicación:**
- **`on`:** Define los eventos que desencadenarán el workflow, en este caso, push y pull_request a la rama `master`.
- **`jobs`:** Define dos trabajos:
  - **`build-and-test`:** 
    - **Checkout Repository:** Clona el repositorio.
    - **Set up Python:** Configura Python 3.9.
    - **Install Dependencies:** Instala las dependencias listadas en `requirements.txt`.
    - **Run Tests:** Ejecuta las pruebas usando PyTest.
  - **`deploy`:**
    - **needs: build-and-test:** Este trabajo solo se ejecuta si `build-and-test` se completa exitosamente.
    - **Checkout Repository:** Clona el repositorio nuevamente.
    - **Login to Heroku Container Registry:** Inicia sesión en Heroku Container Registry usando credenciales almacenadas en secretos de GitHub.
    - **Build, Push and Release Docker image to Heroku:** Empuja la imagen Docker al registro de Heroku y la libera.

### **2.2. Configurar Secretos en GitHub para Heroku**

**Secretos Necesarios:**
- **`HEROKU_EMAIL`:** Correo electrónico asociado a tu cuenta de Heroku.
- **`HEROKU_API_KEY`:** Clave API de Heroku para autenticación.

**Configuración:**
1. Ve a tu repositorio en GitHub.
2. Haz clic en `Settings` > `Secrets and variables` > `Actions`.
3. Haz clic en `New repository secret`.
4. Añade dos secretos:
   - **`HEROKU_EMAIL`:** Tu correo electrónico de Heroku.
   - **`HEROKU_API_KEY`:** Tu API Key de Heroku.

**Nota:** Nunca expongas tus secretos directamente en el código. Utiliza siempre los secretos de GitHub para mantener la seguridad.

### **2.3. Validar y Desplegar Automáticamente**

Con el workflow configurado, cada vez que hagas un push o una pull request a la rama `master`, se ejecutarán las pruebas y, si todo es exitoso, la aplicación se desplegará automáticamente en Heroku.

**Proceso Automatizado:**
1. **Commit y Push de los Cambios:**
   ```bash
   git add .
   git commit -m "Implementar pruebas automatizadas y configurar CI/CD pipeline"
   git push origin master
   ```

2. **Verificar el Workflow en GitHub Actions:**
   - Ve a la pestaña `Actions` en tu repositorio de GitHub.
   - Verifica que el workflow `CI/CD Pipeline` se esté ejecutando y que todas las etapas se completen exitosamente.

3. **Verificar el Despliegue en Heroku:**
   - Una vez que el despliegue haya finalizado, abre tu aplicación en Heroku:
     ```bash
     heroku open --app mnist-flask-api
     ```
   - Envía una solicitud POST a la API desplegada para asegurar que todo funciona correctamente:
     ```bash
     curl -X POST -F file=@ruta/a/tu/imagen.png -H "x-api-key: e3b0c44298fc1c149afbf4c8996fb924" https://mnist-flask-api.herokuapp.com/predict
     ```
     **Respuesta Esperada:**
     ```json
     {
         "prediccion": 5,
         "probabilidad": 0.95
     }
     ```

---

## **3. Verificación del Pipeline de CI/CD**

### **3.1. Probar Cambios en el Código**
- **Proceso:**
  - Realizar cambios menores en el código, como modificar un mensaje en el endpoint `/`.
  - Hacer commit y push de los cambios.
  - Verificar que GitHub Actions ejecuta el workflow: construye, prueba y despliega automáticamente.

### **3.2. Revisar Logs y Despliegues**
- **Heroku Logs:**
  ```bash
  heroku logs --tail --app mnist-flask-api
  ```
- **Verificación:**
  - Asegurarse de que la aplicación está funcionando correctamente y que no hay errores en los logs.

### **3.3. Realizar Pruebas Adicionales**
- **Envía solicitudes a la API y verifica que las predicciones sean correctas.**
- **Introduce escenarios de prueba para asegurar que la autenticación y la validación de entradas funcionen correctamente.**
  ```bash
  curl -X POST -F file=@ruta/a/tu/imagen.png -H "x-api-key: e3b0c44298fc1c149afbf4c8996fb924" https://mnist-flask-api.herokuapp.com/predict
  ```

---

## **4. Conclusiones y Recomendaciones**

### **4.1. Implementación de Pruebas Automatizadas**
- **Beneficios:**
  - **Detección Precoz de Errores:** Identifica problemas antes de desplegar.
  - **Refactorización Segura:** Permite modificar el código con confianza.
  - **Documentación del Comportamiento:** Las pruebas sirven como una forma de documentación del comportamiento esperado de la API.

- **Recomendaciones:**
  - **Cobertura de Pruebas:** Asegúrate de cubrir diferentes escenarios y casos límite en tus pruebas.
  - **Actualización de Pruebas:** Mantén las pruebas actualizadas con cada cambio en la lógica de la aplicación.

### **4.2. Configuración de CI/CD con GitHub Actions**
- **Beneficios:**
  - **Automatización:** Reduce tareas manuales al automatizar pruebas y despliegues.
  - **Consistencia:** Asegura que cada despliegue siga los mismos pasos, minimizando errores humanos.
  - **Rapidez:** Permite desplegar cambios rápidamente una vez que pasan las pruebas.

- **Recomendaciones:**
  - **Monitorear Workflows:** Revisa regularmente el estado de los workflows para detectar y resolver fallos.
  - **Mejorar los Workflows:** Añade más etapas al pipeline, como análisis estático de código o pruebas de integración, según sea necesario.

### **4.3. Mejora Continua**
- **Prácticas Recomendadas:**
  - **Iterar sobre las Pruebas:** A medida que añadas nuevas funcionalidades, incorpora nuevas pruebas para cubrirlas.
  - **Optimizar el Pipeline:** Ajusta el pipeline de CI/CD para que sea más eficiente, por ejemplo, usando cachés para acelerar la instalación de dependencias.
  - **Escalar la Aplicación:** Conforme aumente el uso de la API, considera escalar los recursos en Heroku para manejar la carga.

---

## **Recursos Adicionales**
- **PyTest Documentation:** [PyTest Docs](https://docs.pytest.org/en/7.1.x/)
- **pytest-flask Documentation:** [pytest-flask Docs](https://pytest-flask.readthedocs.io/en/latest/)
- **GitHub Actions Documentation:** [GitHub Actions Docs](https://docs.github.com/en/actions)
- **Heroku Container Registry:** [Heroku Container Registry](https://devcenter.heroku.com/articles/container-registry-and-runtime)
- **Gunicorn Documentation:** [Gunicorn Docs](https://docs.gunicorn.org/en/stable/)
- **Loguru Documentation:** [Loguru Docs](https://loguru.readthedocs.io/en/stable/)
- **Prometheus Documentation:** [Prometheus Docs](https://prometheus.io/docs/introduction/overview/)
- **Grafana Documentation:** [Grafana Docs](https://grafana.com/docs/grafana/latest/)
- **Flask-Swagger UI:** [Flask-Swagger UI GitHub](https://github.com/sveintsev/flask-swagger-ui)

---

## **Conclusión**

En el **Día 21**, has avanzado significativamente al **implementar pruebas automatizadas** y **configurar un pipeline de CI/CD** para tu API de predicción de dígitos MNIST. Estas prácticas son esenciales para garantizar la calidad, confiabilidad y eficiencia de tu aplicación en entornos de producción.

**Pasos Clave Realizados:**
1. **Implementación de Pruebas Automatizadas:** Escribiste pruebas unitarias e integradas para verificar el funcionamiento correcto de la API.
2. **Configuración de CI/CD con GitHub Actions:** Automatizaste el proceso de pruebas y despliegue, asegurando que cada cambio en el código pase por pruebas antes de ser desplegado.
3. **Integración con Heroku:** Configuraste el despliegue automático en Heroku, permitiendo que los cambios se publiquen en producción de manera eficiente y segura.

**Recomendaciones para Continuar:**
- **Expandir las Pruebas:** Añade más pruebas para cubrir nuevos endpoints o funcionalidades.
- **Mejorar el Pipeline de CI/CD:** Incorpora etapas adicionales, como análisis de código estático o pruebas de integración.
- **Optimizar el Rendimiento y Escalabilidad:** Monitorea el rendimiento de la API y ajusta los recursos en Heroku según sea necesario.
- **Automatizar la Actualización del Modelo:** Implementa un flujo de trabajo para actualizar el modelo de machine learning con nuevos datos de manera automatizada.

¡Has realizado un excelente progreso en tu proyecto de **Transfer Learning**! Continúa aplicando estas prácticas en tus futuros proyectos para mantener altos estándares de calidad y eficiencia.

Si tienes alguna otra pregunta o necesitas más asistencia en los próximos pasos, **no dudes en contactarme**. ¡Estoy aquí para ayudarte a alcanzar el éxito en tu proyecto de **Transfer Learning**!